In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from scipy.interpolate import griddata

# 1. Load the HYCOM dataset from the specified path
hycom_ds = xr.open_dataset(r"C:\Users\soele\OneDrive - George Mason University - O365 Production\Documents\FINESST\Paper-3\ssh_2024.nc4")

# Check if scale_factor exists, otherwise assume it's 1
scale_factor = hycom_ds['surf_el'].attrs.get('scale_factor', 1.0)  # Default to 1.0 if missing

# Apply scaling factor (if any)
hycom_surf_el = hycom_ds['surf_el'] * scale_factor
hycom_lon = hycom_ds['lon'].values
hycom_lat = hycom_ds['lat'].values

# 2. Convert HYCOM longitudes from 0-360 to -180 to 180
hycom_lon = np.where(hycom_lon > 180, hycom_lon - 360, hycom_lon)

# Check for the presence of _FillValue, if not found assume no fill value
fill_value = hycom_ds['surf_el'].attrs.get('_FillValue', np.nan)

# Mask HYCOM data for missing values
hycom_surf_el_cleaned = hycom_surf_el.where(hycom_surf_el != fill_value)

# 3. Load the ADCIRC dataset from the specified path
adcirc_ds = xr.open_dataset(r"C:\Users\soele\OneDrive - George Mason University - O365 Production\Documents\FINESST\Paper-3\fort_test.63.nc")

# Extract the 585th timestep (which corresponds to the same time as the HYCOM data)
adcirc_zeta = adcirc_ds['zeta'].isel(time=585)  # Only load the 585th timestep
adcirc_lon = adcirc_ds['x'].values
adcirc_lat = adcirc_ds['y'].values

# Mask ADCIRC data for missing values
adcirc_fill_value = adcirc_ds['zeta'].attrs.get('_FillValue', np.nan)
adcirc_zeta_cleaned = adcirc_zeta.where(adcirc_zeta != adcirc_fill_value).values

# 4. Interpolate ADCIRC data to HYCOM grid using scipy's griddata
adcirc_interpolated = griddata(
    (adcirc_lon, adcirc_lat),  # ADCIRC grid
    adcirc_zeta_cleaned,       # ADCIRC zeta values
    (hycom_lon[None, :], hycom_lat[:, None]),  # HYCOM grid for interpolation
    method='linear'
)

# Handle any out-of-bounds interpolation results (e.g., NaNs)
adcirc_interpolated = np.ma.masked_invalid(adcirc_interpolated)

# 5. Compute the difference (HYCOM - ADCIRC)
difference = hycom_surf_el_cleaned.isel(time=0).values - adcirc_interpolated

# 6. Plot the difference
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

# Contour plot of the difference
contour = ax.contourf(hycom_lon, hycom_lat, difference, 60, cmap='coolwarm', transform=ccrs.PlateCarree())
plt.colorbar(contour, label='Difference in Water Surface Elevation (m)')

# Add coastlines for context
ax.coastlines()

land = cfeature.NaturalEarthFeature('physical', 'land', '110m', edgecolor='face', facecolor='darkgray')
ax.add_feature(land)

# Add title and labels
plt.title('Diff of HYCOM and ADCIRC WSE (August 25, 2024, 9am EST)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()